In [1]:
!pip install mip

## LEF DEF Parsing

- Install the LEFDEFParser from the the wheel file : [LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl](https://github.com/srini229/EE5333_tutorials/blob/master/parser/LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl)
- Download example LEF and DEF files: [Nangate.lef](https://github.com/srini229/EE5333_tutorials/blob/master/parser/Nangate.lef) and [example.def](https://github.com/srini229/EE5333_tutorials/blob/master/parser/example.def)

    <img src="https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/part/fig/example_cir.png" width=340 height=195 />



In [2]:
!pip install --break-system-packages https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/LEFDEFParser-0.1-cp310-cp310-linux_x86_64.whl
!rm -f *.{lef,def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/{Nangate.lef,example.def}
!wget https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/sample.{lef,def}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 8.6 MB/s eta 0:00:00
--2024-02-13 04:33:34--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/Nangate.lef
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1083933 (1.0M) [text/plain]
Saving to: ‘Nangate.lef’

Nangate.lef         100%[===================>]   1.03M  --.-KB/s    in 0.06s   

2024-02-13 04:33:35 (18.3 MB/s) - ‘Nangate.lef’ saved [1083933/1083933]

--2024-02-13 04:33:35--  https://raw.githubusercontent.com/srini229/EE5333_tutorials/master/parser/example.def
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 899 [text/plain]
Saving to: ‘example.def’

example.def         100%[===================>]  

In [3]:
#vertex class; name is the instance name of the gate; area: area of the module and the;
class Vertex:
  def __init__(self, name, area):
    self._name = name
    self._area = area
  def __repr__(self):
    return self._name + f" ({round(self._area,2)})"

# Reads input LEF/DEF files and returns a hypergraph with vertex list V and hyperedge list E;
# each hyperedge in E is a tuple of vertices of type `Vertex` defined above.
def loadNetlist(leffile = None, deffile = None):
  import LEFDEFParser as LDP
  l = LDP.LEFReader()
  areaLookup = dict()
  if leffile:
    l.readLEF(leffile)
    areaLookup = {m.name():(m.xdim()*m.ydim()*1.e-6) for m in l.macros()}
  vertices = dict()
  edges = dict()
  if deffile:
    d = LDP.DEFReader()
    d.readDEF(deffile)
    vertices = {c.name() : Vertex(c.name(), areaLookup.get(c.macro(), None)) for c in d.components()}
    edges = {n.name():[vertices[p[0]] for p in n.pins() if p[0] != 'PIN'] for n in d.nets()}
  delE = list()
  for e in edges:
    if len(edges[e]) <= 1:
      delE.append(e)
  for e in delE: del edges[e]
  return vertices, edges

## $k$-way hypergraph partitioning using ILP
+ Hypergraph $H(V,E)$
+ $x_{v,i}$ is the indicator variable for $v$ being in partition $V_i$
+ $x_{e,i}$ is the indicator variable for $e\in E$ being contained in $V_i$

+ Objective: $\max\limits_{x_{e,i}} \sum\limits_{e\in E}\sum\limits_{i=1}^k x_{e,i}$
+ Subject to constraints:
<ul>
$\begin{align}
x_{v,i} &\in \{0, 1\}, &\forall v \in V, \forall i \in \{1,2,\ldots, k\}\\
x_{e,i} &\in \{0, 1\}, &\forall e \in E, \forall i \in \{1,2,\ldots, k\}\\
\sum\limits_{i=1}^k x_{v,i} &=1 , &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\leq Area_{max} &\forall v \in V\\
\sum_{v\in V} area(v)\cdot x_{v,i}&\geq Area_{min} &\forall v \in V\\
x_{e,i} &\leq x_{v,i}, &\forall e \in E, \forall v~\text{connected by}~e\\
\end{align}$
</ul>

In [4]:
# V - vertices; each vertex is one of the gates in the netlist; Each vertex is of type
# E - hyperedges; each edge/hyperedge is a tuple of vertices
# k - is the number of partitions
# (Amin, Amax) - minimum/maximum area for balance criterion
def partition(V, E, Amin, Amax, k = 2):
  import mip
  model = mip.Model(f"{k}-way partition")
  x = {u:[model.add_var(f"x_{u}_{i}", var_type = mip.BINARY) for i in range(k)] for u in V}
  x_e = {e:[model.add_var(f"x_{e}_{i}", var_type = mip.BINARY) for i in range(k)] for e in E}
  model.verbose = 0
  model.objective = mip.maximize(mip.xsum(x_e[e][i] for e in E for i in range(k)))

  for u in V:
    model += mip.xsum(x[u]) == 1

  for i in range(k):
    model += mip.xsum(V[u]._area*x[u][i] for u in V) >= Amin
    model += mip.xsum(V[u]._area*x[u][i] for u in V) <= Amax

  for e in E:
    for i in range(k):
      for v in E[e]:
        model += x_e[e][i] <= x[v._name][i]

  model.write(f"partition{k}.lp")
  model.optimize()
  sol = [list() for i in range(k)]

  if model.status == mip.OptimizationStatus.OPTIMAL:
    for u in V:
      for i in range(k):
        if round(x[u][i].x) == 1: sol[i].append(V[u])

    return (sol, len(E)- model.objective.x)
  return None, None


In [5]:
# bi-partition the input hypergraph using Fiduccia-Matheyses algorithm
# argument list is the same as the ILP version described above
# return value is a list of two lists and the number of cut hyperedges;
# each list is a partition comprising contained gates(vertices)
def partitionFM(V, E, Amin, Amax):
  return None, None

In [6]:
import time
for (lf,df) in [('sample.lef', 'sample.def'), ('Nangate.lef', 'example.def')]:
  V,E = loadNetlist(lf, df)
  Atotal = sum(V[u]._area for u in V)
  maxCellArea = max(V[u]._area for u in V)
  print("Total area :", Atotal, "delArea :", round(maxCellArea,2))
  for partfn in [partition, partitionFM]:
    t = time.time()
    sol, numcuts = partfn(V, E, Atotal/2 - maxCellArea, Atotal/2 + maxCellArea)
    print(f"{partfn.__name__} runtime : {time.time() - t}")
    if sol:
      print("number of cuts :", round(numcuts))
      for part in sol:
        print("area :", round(sum([x._area for x in part]),2), part)

Total area : 250.344 delArea : 25.99
partition runtime : 1.878967523574829
number of cuts : 0
area : 142.27 [inst2591 (15.05), inst2908 (9.58), inst3428 (8.21), inst3444 (8.21), inst4062 (12.31), inst4678 (5.47), inst4189 (15.05), inst4382 (8.21), inst5638 (12.31), inst5275 (9.58), inst5821 (6.84), inst6050 (8.21), inst5195 (12.31), inst7234 (10.94)]
area : 108.07 [inst2015 (10.94), inst3502 (13.68), inst4132 (8.21), inst4183 (25.99), inst4597 (8.21), inst5333 (12.31), inst6286 (10.94), inst6458 (17.78)]
partitionFM runtime : 7.152557373046875e-06
Total area : 18.088 delArea : 3.19
partition runtime : 0.0233917236328125
number of cuts : 2
area : 9.58 [a (3.19), c (3.19), e (3.19)]
area : 8.51 [b (3.19), d (3.19), f (2.13)]
partitionFM runtime : 5.4836273193359375e-06
